<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Sentiment Analysis using UDF</b>
</header>

### Disclaimer
The sample code (“Sample Code”) provided is not covered by any Teradata agreements. Please be aware that Teradata has no control over the model responses to such sample code and such response may vary. The use of the model by Teradata is strictly for demonstration purposes and does not constitute any form of certification or endorsement. The sample code is provided “AS IS” and any express or implied warranties, including the implied warranties of merchantability and fitness for a particular purpose, are disclaimed. In no event shall Teradata be liable for any direct, indirect, incidental, special, exemplary, or consequential damages (including, but not limited to, procurement of substitute goods or services; loss of use, data, or profits; or business interruption) sustained by you or a third party, however caused and on any theory of liability, whether in contract, strict liability, or tort arising in any way out of the use of this sample code, even if advised of the possibility of such damage.

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Usecase</b><br>
Create a user defined function to determine whether the reviews of the product provided by the customer in the given dataset is positive, negative or neutral using VADER Sentiment Analysis Tool.

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Import the required libraries</b>

In [2]:
# Importing required libraries
import getpass
from teradataml import *
from teradataml.dataframe.functions import udf
from teradatasqlalchemy.types import VARCHAR

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Connect to Vantage</b>

In [3]:
# Create the connection.
td_context = create_context(host=getpass.getpass("Hostname: "), username=getpass.getpass("Username: "), password=getpass.getpass("Password: "))

Hostname:  ········
Username:  ········
Password:  ········


In [4]:
# Set the Authentication Token
set_config_params(auth_token=getpass.getpass("auth_data: "), ues_url=getpass.getpass("ues_url: "))

auth_data:  ········
ues_url:  ········


True

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Loading Dataset</b>

In [5]:
# Load the example dataset.
load_example_data("sentimentextractor", "sentiment_extract_input")
df = DataFrame("sentiment_extract_input")

In [6]:
df

id,product,category,review
4,gps,pos,"it is a fine gps . outstanding performance , works great . you can even get incredible coordinate accuracy from streets and trips to compare ."
8,camera,neg,"i hate my camera , and im stuck with it . this camera sucks so bad , even the dealers on ebay have difficulty selling it. horrible indoors , does not capture fast action, screwy software , no suprise , and screwy audio/video codec that does not work with hardly any app"
6,gps,neg,most of the complaints i have seen in here are from a lack of rtfm. i have never seen so many mistakes do to what i think has to be none update of data to the system . i wish i could make all the rating stars be empty .
5,gps,pos,"nice graphs and map route info .i would not run outside again without this unique gadget . great job. big display , good backlight , really watertight , training assistant .i use in trail running and it worked well through out the race"
3,camera,pos,"this is a nice camera , delivering good quality video images decent photos ."
10,camera,neg,"i returned my camera to the vendor as i will not tolerate a sub standard product that is a known issue especially from vendor who will not admt that this needs to be removed from the shelf due to failing parts updated . due to the constant need for repair , i would never recommend this product ."
2,office suite,pos,"it is the best office suite i have used to date . it is launched before office 2010 and it is ages ahead of it already . the fact that i could comfortable import xls , doc , ppt and modify them , and then export them back to the doc , xls , ppt is terrific . i needed the compatibility .it is a very intuitive suite and the drag drop functionality is terrific ."
1,camera,pos,"we primarily bought this camera for high image quality and excellent video capability without paying the price for a dslr .it has excelled in what we expected of it , and consequently represented excellent value for me .all my friends want my camera for their vacations . i would recommend this camera to anybody .definitely worth the price .plus , when you buy some accessories , it becomes even more powerful"
7,gps,neg,this machine is all screwed up . on my way home from a friends house it told me there is no possible route . i found their website support difficult to navigate . i am is so disapointed and just returned it and now looking for another one
9,television,neg,"$3k is way too much money to drop onto a piece of crap .poor customer support . after about 1 and a half years and hardly using the tv , a big yellow pixilated stain appeared. product is very inferior and subject to several lawsuits . i expressed my dissatifaction with the situation as this is a known issue"


<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Setting up the Environment</b>

In [7]:
list_user_envs()

,env_name,env_description,base_env_name,language,conda
0,conda_env_3_10_demo,Conda environment for notebook demo,python_3.10,python,True
1,demo_env,Demo env 1.,python_3.10,Python,False
2,non_conda_env_3_8_demo,Non Conda environment for notebook demo,python_3.8,Python,False
3,openml_env,DONT DELETE: OpenML environment,python_3.10,Python,False
4,openml_env_dhan,DONT DELETE: OpenML environment,python_3.10,Python,False
5,testenv,This env 'testenv' is created with base env 'p...,python_3.10,Python,False
6,testudf,Test environment for UDF,python_3.10,Python,False


In [8]:
# create a new environment use create_env function, if environment is already created use get_env to get access to the environment.
env = create_env('text_analysis', 'python_3.10', 'Test environment for UDF')

User environment 'text_analysis' created.


In [9]:
# Install the required package to run the user defined function in the user environment using install_lib function Of UserEnv class.
env.install_lib('vaderSentiment')

,Claim Id,File/Libs/Model,Method Name,Stage,Timestamp,Additional Details
0,68d0071a-0871-4425-a43d-260f360df6f7,vaderSentiment,install_lib,Started,2024-09-04T10:14:34Z,
1,68d0071a-0871-4425-a43d-260f360df6f7,vaderSentiment,install_lib,Finished,2024-09-04T10:14:40Z,


In [10]:
env.libs

,name,version
0,certifi,2024.8.30
1,charset-normalizer,3.3.2
2,idna,3.8
3,pip,23.0.1
4,requests,2.32.3
5,setuptools,65.5.0
6,urllib3,2.2.2
7,vaderSentiment,3.3.2


<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Create a UDF to determine whether the values in 'review' column is positive, negative or neutral.</b>

udf to determine the sentiment of the values in 'review' column. As the data in 'review' column contains `,`  character which is same as the default delimiter, so here passing the different delimiter `|`.

In [11]:
@udf(env_name = 'text_analysis', returns = VARCHAR(80),  delimiter='|')
def sentiment_analysis(txt):
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(txt)

    if sentiment_dict['compound'] >= 0.05 :
        sentiment = "Positive"
    elif sentiment_dict['compound'] <= - 0.05 :
        sentiment = "Negative"
    else :
        sentiment = "Neutral"
    return sentiment

In [12]:
#Assign the Column Expression returned by user defined function to the DataFrame.
out_df = df.assign(sentiment = sentiment_analysis('review'))

# Print the result.
out_df

id,product,category,review,sentiment
6,gps,neg,most of the complaints i have seen in here are from a lack of rtfm. i have never seen so many mistakes do to what i think has to be none update of data to the system . i wish i could make all the rating stars be empty .,Negative
2,office suite,pos,"it is the best office suite i have used to date . it is launched before office 2010 and it is ages ahead of it already . the fact that i could comfortable import xls , doc , ppt and modify them , and then export them back to the doc , xls , ppt is terrific . i needed the compatibility .it is a very intuitive suite and the drag drop functionality is terrific .",Positive
9,television,neg,"$3k is way too much money to drop onto a piece of crap .poor customer support . after about 1 and a half years and hardly using the tv , a big yellow pixilated stain appeared. product is very inferior and subject to several lawsuits . i expressed my dissatifaction with the situation as this is a known issue",Negative
7,gps,neg,this machine is all screwed up . on my way home from a friends house it told me there is no possible route . i found their website support difficult to navigate . i am is so disapointed and just returned it and now looking for another one,Negative
1,camera,pos,"we primarily bought this camera for high image quality and excellent video capability without paying the price for a dslr .it has excelled in what we expected of it , and consequently represented excellent value for me .all my friends want my camera for their vacations . i would recommend this camera to anybody .definitely worth the price .plus , when you buy some accessories , it becomes even more powerful",Positive
8,camera,neg,"i hate my camera , and im stuck with it . this camera sucks so bad , even the dealers on ebay have difficulty selling it. horrible indoors , does not capture fast action, screwy software , no suprise , and screwy audio/video codec that does not work with hardly any app",Negative
5,gps,pos,"nice graphs and map route info .i would not run outside again without this unique gadget . great job. big display , good backlight , really watertight , training assistant .i use in trail running and it worked well through out the race",Positive
3,camera,pos,"this is a nice camera , delivering good quality video images decent photos .",Positive
4,gps,pos,"it is a fine gps . outstanding performance , works great . you can even get incredible coordinate accuracy from streets and trips to compare .",Positive
10,camera,neg,"i returned my camera to the vendor as i will not tolerate a sub standard product that is a known issue especially from vendor who will not admt that this needs to be removed from the shelf due to failing parts updated . due to the constant need for repair , i would never recommend this product .",Negative


<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Disconnect from Vantage</b>

In [13]:
remove_context()

True